### Poeem search

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

: 

In [33]:
pid = os.getpid()
!kill -9 $pid

: 

: 

In [3]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import poeem

In [4]:
EMB_DIM = 64

#### Movielens

In [5]:
import pandas as pd

In [6]:
data_rate = pd.read_csv('../data/ratings.csv')
data_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


In [6]:
print('max:{}'.format(data_rate['userId'].max()), 'min:{}'.format(data_rate['userId'].min()), data_rate['userId'].value_counts())
print('max:{}'.format(data_rate['movieId'].max()), 'min:{}'.format(data_rate['movieId'].min()), data_rate['movieId'].value_counts())
print('max:{}'.format(data_rate['rating'].max()), 'min:{}'.format(data_rate['rating'].min()), data_rate['rating'].value_counts())

max:138493 min:1 118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
          ... 
89305       20
110463      20
96990       20
134747      20
6526        20
Name: userId, Length: 138493, dtype: int64
max:131262 min:1 296       67310
356       66172
318       63366
593       63299
480       59715
          ...  
125545        1
78873         1
112907        1
112909        1
110510        1
Name: movieId, Length: 26744, dtype: int64
max:5.0 min:0.5 4.0    5561926
3.0    4291193
5.0    2898660
3.5    2200156
4.5    1534824
2.0    1430997
2.5     883398
1.0     680732
1.5     279252
0.5     239125
Name: rating, dtype: int64


In [76]:
N_all = len(data_rate)
user_all = list(data_rate['userId'].unique())
movie_all = list(data_rate['movieId'].unique())
N_user, N_movie = len(user_all), len(movie_all)
N_all, N_user, N_movie

(20000263, 138493, 26744)

In [14]:
user_all = list(data_rate['userId'].unique())
movie_all = list(data_rate['movieId'].unique())
len(user_all), len(movie_all)

(138493, 26744)

In [29]:
# def split_stratified_into_train_val_test(df_input, stratify_colname='y',
#                     frac_train=0.6, frac_val=0.2, frac_test=0.2,
#                     random_state=None):
#     '''
#     Splits a Pandas dataframe into three subsets (train, val, and test)
#     following fractional ratios provided by the user, where each subset is
#     stratified by the values in a specific column (that is, each subset has
#     the same relative frequency of the values in the column). It performs this
#     splitting by running train_test_split() twice.

#     Parameters
#     ----------
#     df_input : Pandas dataframe
#         Input dataframe to be split.
#     stratify_colname : str
#         The name of the column that will be used for stratification. Usually
#         this column would be for the label.
#     frac_train : float
#     frac_val   : float
#     frac_test  : float
#         The ratios with which the dataframe will be split into train, val, and
#         test data. The values should be expressed as float fractions and should
#         sum to 1.0.
#     random_state : int, None, or RandomStateInstance
#         Value to be passed to train_test_split().

#     Returns
#     -------
#     df_train, df_val, df_test :
#         Dataframes containing the three splits.
#     '''

#     if frac_train + frac_val + frac_test != 1.0:
#         raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
#                          (frac_train, frac_val, frac_test))

#     if stratify_colname not in df_input.columns:
#         raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

#     X = df_input # Contains all columns.
#     y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

#     # Split original dataframe into train and temp dataframes.
#     df_train, df_temp, y_train, y_temp = train_test_split(X,
#                                 y,
#                                 stratify=y,
#                                 test_size=(1.0 - frac_train),
#                                 random_state=random_state)

#     # Split the temp dataframe into val and test dataframes.
#     relative_frac_test = frac_test / (frac_val + frac_test)
#     df_val, df_test, y_val, y_test = train_test_split(df_temp,
#                                 y_temp,
#                                 stratify=y_temp,
#                                 test_size=relative_frac_test,
#                                 random_state=random_state)

#     assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

#     return df_train, df_val, df_test

In [30]:
# data_train, data_val, data_test = split_stratified_into_train_val_test(data_rate, 'userId')

In [ ]:
def check_contains_all(df, user_all, movie_all):
    user_list, item_list = [], []
    user_unique, movie_unique = list(df['userId'].unique()), list(df['movieId'].unique())
    # for user in  user_all:
    #     if user not in user_unique:
    #         user_list.append(user)
    #         # print('userId:{}'.format(user))
    #         # return False
    # for movie in movie_all:
    #     if movie not in movie_unique:
    #         item_list.append(movie)
    #         # print('movie:{}'.format(movie))
    #         # return False
    # # return True
    # return user_list, item_list
    return len(user_all)-len(user_unique), len(movie_all)-len(movie_unique)

In [66]:
cnt = 0
while cnt<100:
    data_train, data_val, data_test = np.split(data_rate.sample(frac=1),
                            [int(.6*len(data_rate)), int(.8*len(data_rate))])
    u, m = check_contains_all(data_train, user_all, movie_all)
    if u+m == 0:
        break
    cnt += 1
    print(cnt)
len(data_train), len(data_val), len(data_test), len(data_rate)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


(12000157, 4000053, 4000053, 20000263)

In [54]:
len(data_rate['userId'].unique()), len(data_train['userId'].unique())

(138493, 138493)

In [71]:
user_left, movie_left = check_contains_all(data_train, user_all, movie_all)
user_left, movie_left

(0, 1957)

In [75]:
data_train.to_csv('../data/ratings_train.csv')
data_test.to_csv('../data/ratings_test.csv')
data_val.to_csv('../data/ratings_val.csv')

In [77]:
N_user, N_movie = 138493, 26744

#### Poeem Model

In [87]:
BATCH_SIZE = 1000
LEARNING_RATE = 0.1
EPOCH = 3
EMB_DIM = 64

In [88]:
def model_fn(features, labels, mode):
    query_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='query',
            vocabulary_list=range(N_user),
            dtype=tf.int32),
        dimension=EMB_DIM)
    item_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='item',
            vocabulary_list=range(N_movie),
            dtype=tf.int32),
        dimension=EMB_DIM)
    
    query_emb = tf.feature_column.input_layer(features, [query_column])
    item_emb = tf.feature_column.input_layer(features, [item_column])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode, predictions={'query': query_emb, 'item': item_emb})

    def cosine(a, b):
        a = tf.nn.l2_normalize(a, axis=1)
        b = tf.nn.l2_normalize(b, axis=1)
        return tf.matmul(a, b, transpose_b=True)

    scores = cosine(query_emb, item_emb)

    batch_size = tf.shape(query_emb)[0]
    loss = tf.reduce_sum(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.eye(batch_size),
            logits=scores * 30))  # 1/30 is softmax temperature. Not carefully tune.

    optimizer = tf.train.AdagradOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(
        mode, loss=loss, train_op=train_op, predictions={'query': query_emb, 'item': item_emb})

In [89]:
def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices({'query': data_train['userId'].to_numpy().astype(np.int32), 
                            'item': data_train['movieId'].to_numpy().astype(np.int32)})
    dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).repeat(EPOCH)
    return dataset

In [93]:
tf.test.is_built_with_cuda()

tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

2023-02-13 18:09:29.732227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:08:00.0
2023-02-13 18:09:29.732704: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64:/usr/lib/x86_64-linux-gnu
2023-02-13 18:09:29.732905: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64:/usr/lib/x86_64-linux-gnu
2023-02-13 18:09:29.733093: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcufft.so.10.0'; dlerror: libcufft.so.10.0: cannot open s

False

In [90]:
retrieval_model = tf.estimator.Estimator(model_fn=model_fn)
retrieval_model.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp5d8_bjg7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faad1dcce10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done c

2023-02-13 18:05:15.350125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:08:00.0
2023-02-13 18:05:15.350599: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64:/usr/lib/x86_64-linux-gnu
2023-02-13 18:05:15.350796: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64:/usr/lib/x86_64-linux-gnu
2023-02-13 18:05:15.350974: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcufft.so.10.0'; dlerror: libcufft.so.10.0: cannot open s

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp5d8_bjg7/model.ckpt.
INFO:tensorflow:loss = 12673.435, step = 1
INFO:tensorflow:global_step/sec: 74.1644
INFO:tensorflow:loss = 12959.156, step = 101 (1.350 sec)
INFO:tensorflow:global_step/sec: 82.3093
INFO:tensorflow:loss = 12405.019, step = 201 (1.215 sec)
INFO:tensorflow:global_step/sec: 80.728
INFO:tensorflow:loss = 11874.441, step = 301 (1.239 sec)
INFO:tensorflow:global_step/sec: 78.6485
INFO:tensorflow:loss = 11310.115, step = 401 (1.272 sec)
INFO:tensorflow:global_step/sec: 72.2063
INFO:tensorflow:loss = 10969.152, step = 501 (1.386 sec)
INFO:tensorflow:global_step/sec: 78.4474
INFO:tensorflow:loss = 10391.017, step = 601 (1.275 sec)
INFO:tensorflow:global_step/sec: 81.43
INFO:tensorflow:loss = 9868.72, step = 701 (1.226 sec)
INFO:tensorflow:global_step/sec: 81.1236
INFO:tensorflow:loss = 9381.947, step = 801 (1.233 sec)
INFO:tensorflow:global_step/sec: 81.5481
INFO:tensorflow:los

KeyboardInterrupt: 

#### Test

In [ ]:
def poeem_model_fn(features, labels, mode, params):
    query_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='query',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    query_emb = tf.feature_column.input_layer(features, [query_column])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        ### [poeem code] directly do ANN search in the model as a TensorFlow op.
        if params.get('item_search', False):
            index = poeem.search.index_from_file(params['index_file'])
            neighbors, scores = index.search(
                tf.expand_dims(query_emb, 1),
                params['topk'],
                params['nprobe'],
                params['metric_type'],
                verbose=False)        
            return tf.estimator.EstimatorSpec(
                mode, predictions={'neighbors': neighbors, 'scores': scores})
        ### end [poeem code]
        
    item_column = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='item',
            vocabulary_list=range(VOCAB_SIZE),
            dtype=tf.int32),
        dimension=EMB_DIM)
    item_emb = tf.feature_column.input_layer(features, [item_column])
    item_emb = tf.nn.l2_normalize(item_emb, axis=1)
    

    ### [poeem code] item indexing layer as the last layer in item tower
    hparams = poeem.embedding.PoeemHparam(coarse_K=8,
                        K=256,
                        D=8,
                        rotate=0) # exactly the same parameters as Faiss, specified above.
    item_batch_quantized = poeem.embedding.PoeemEmbed(
        EMB_DIM,
        warmup_steps=16384,
        buffer_size=8192,
        hparams=hparams,
        mode=mode)
    
    # gradient straight-through estimator. For details, check out our paper.
    item_emb_tau, coarse_code, code, regularizer = item_batch_quantized.forward(item_emb)
    item_emb = item_emb - tf.stop_gradient(item_emb - item_emb_tau)
    ### end [poeem code] 

    if mode == tf.estimator.ModeKeys.PREDICT:
        ### [poeem code] exprt item embeddings/PQ code for disk persistency.
        if params.get('item_predict', False):
            return tf.estimator.EstimatorSpec(
                mode, predictions={
                    'item_coarse_code': coarse_code,
                    'item_code': code,
                    'item_norm': tf.norm(item_emb, axis=1)
                })
        ### end [poeem code] 

    def cosine(a, b):
        a = tf.nn.l2_normalize(a, axis=1)
        b = tf.nn.l2_normalize(b, axis=1)
        return tf.matmul(a, b, transpose_b=True)

    scores = cosine(query_emb, item_emb)

    batch_size = tf.shape(query_emb)[0]
    loss = tf.reduce_sum(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.eye(batch_size),
            logits=scores * 30))
    
    loss = loss + regularizer

    optimizer = tf.train.AdagradOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(
        mode, loss=loss, train_op=train_op, predictions={'query': query_emb, 'item': item_emb})
